In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 0)
RANDOM_STATE = 11

In [2]:
df = pd.read_csv('../data/train.csv')
print(df.shape)
df.head(5)

(58592, 44)


,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,max_torque,max_power,engine_type,airbags,is_esc,is_adjustable_steering,is_tpms,is_parking_sensors,is_parking_camera,rear_brakes_type,displacement,cylinder,transmission_type,gear_box,steering_type,turning_radius,length,width,height,gross_weight,is_front_fog_lights,is_rear_window_wiper,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,2,No,No,No,Yes,No,Drum,796,3,Manual,5,Power,4.6,3445,1515,1475,1185,No,No,No,No,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,2,No,No,No,Yes,No,Drum,796,3,Manual,5,Power,4.6,3445,1515,1475,1185,No,No,No,No,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,60Nm@3500rpm,40.36bhp@6000rpm,F8D Petrol Engine,2,No,No,No,Yes,No,Drum,796,3,Manual,5,Power,4.6,3445,1515,1475,1185,No,No,No,No,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,113Nm@4400rpm,88.50bhp@6000rpm,1.2 L K12N Dualjet,2,Yes,Yes,No,Yes,Yes,Drum,1197,4,Automatic,5,Electric,4.8,3995,1735,1515,1335,Yes,No,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,91Nm@4250rpm,67.06bhp@5500rpm,1.0 SCe,2,No,No,No,No,Yes,Drum,999,3,Automatic,5,Electric,5.0,3731,1579,1490,1155,No,No,No,No,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0


In [3]:
df.isna().sum(axis=0).sum()

0

## train test split

In [4]:
import numpy as np
from toolbox import train_test_split

In [5]:
y = df.pop('is_claim')
train_X, test_X, train_y, test_y= train_test_split(df, y, random_state=RANDOM_STATE, train_size=0.8)
print('{:<10} :'.format('train_X shape'), train_X.shape)
print('{:<10} :'.format('train_y shape'), train_y.shape)
print('-'*20)
print('{:<10} :'.format('test_X shape'), test_X.shape)
print('{:<10} :'.format('test_y shape'), test_y.shape)

train_X shape : (46874, 43)
train_y shape : (46874,)
--------------------
test_X shape : (11718, 43)
test_y shape : (11718,)


## preprocessing

In [6]:
num_col = []
cat_col = []
for col, dtype in train_X.dtypes.items():
    if str(dtype) == 'object':
        cat_col.append(col)
    else:
        num_col.append(col)
print('num_col: ', num_col)
print('-'*10)
print('cat_col: ', cat_col)

num_col:  ['policy_tenure', 'age_of_car', 'age_of_policyholder', 'population_density', 'make', 'airbags', 'displacement', 'cylinder', 'gear_box', 'turning_radius', 'length', 'width', 'height', 'gross_weight', 'ncap_rating']
----------
cat_col:  ['policy_id', 'area_cluster', 'segment', 'model', 'fuel_type', 'max_torque', 'max_power', 'engine_type', 'is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors', 'is_parking_camera', 'rear_brakes_type', 'transmission_type', 'steering_type', 'is_front_fog_lights', 'is_rear_window_wiper', 'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist', 'is_power_door_locks', 'is_central_locking', 'is_power_steering', 'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert']


### tool

In [7]:
from toolbox import one_hot_encode
from toolbox import target_encode
from toolbox import yes_no_01

# one_hot_encode(train_X, ['make'])
# target_encode(train_X, train_y, 'model')

### num col

In [8]:
train_X[num_col].describe()

,policy_tenure,age_of_car,age_of_policyholder,population_density,make,airbags,displacement,cylinder,gear_box,turning_radius,length,width,height,gross_weight,ncap_rating
count,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000,46874.000000
mean,0.610754,0.069516,0.469385,18852.154627,1.766160,3.135598,1162.891667,3.628131,5.244912,4.853189,3850.873555,1672.453620,1553.170628,1385.108888,1.763600
std,0.414248,0.056805,0.123072,17680.818767,1.139398,1.831801,266.128959,0.483309,0.430040,0.227770,311.027018,111.974441,79.477919,212.348698,1.391075
min,0.002735,0.000000,0.288462,290.000000,1.000000,1.000000,796.000000,3.000000,5.000000,4.500000,3445.000000,1475.000000,1475.000000,1051.000000,0.000000
25%,0.209721,0.020000,0.365385,6112.000000,1.000000,2.000000,796.000000,3.000000,5.000000,4.600000,3445.000000,1515.000000,1475.000000,1185.000000,0.000000
50%,0.572717,0.060000,0.451923,8794.000000,1.000000,2.000000,1197.000000,4.000000,5.000000,4.800000,3845.000000,1735.000000,1530.000000,1335.000000,2.000000
75%,1.038918,0.110000,0.548077,27003.000000,3.000000,6.000000,1493.000000,4.000000,5.000000,5.000000,3995.000000,1755.000000,1635.000000,1510.000000,3.000000
max,1.396641,1.000000,1.000000,73430.000000,5.000000,6.000000,1498.000000,4.000000,6.000000,5.200000,4300.000000,1811.000000,1825.000000,1720.000000,5.000000


* onehot encoding： make
* cylinder -> cylinder_4 (0, 1
* gear_box -> gear_box_6 (0, 1

In [9]:
train_X['cylinder_4'] = 0
mask = (train_X['cylinder'] == 4)
train_X.loc[mask, 'cylinder_4'] = 1

train_X['gear_box_6'] = 0
mask = (train_X['gear_box'] == 6)
train_X.loc[mask, 'gear_box_6'] = 1

train_X = train_X.drop(['cylinder', 'gear_box'], axis=1)
# --- test ---
test_X['cylinder_4'] = 0
mask = (test_X['cylinder'] == 4)
test_X.loc[mask, 'cylinder_4'] = 1

test_X['gear_box_6'] = 0
mask = (test_X['gear_box'] == 6)
test_X.loc[mask, 'gear_box_6'] = 1
test_X = test_X.drop(['cylinder', 'gear_box'], axis=1)

### cat col

In [10]:
train_X[cat_col].describe()

,policy_id,area_cluster,segment,model,fuel_type,max_torque,max_power,engine_type,is_esc,is_adjustable_steering,is_tpms,is_parking_sensors,is_parking_camera,rear_brakes_type,transmission_type,steering_type,is_front_fog_lights,is_rear_window_wiper,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert
count,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874,46874
unique,46874,22,6,11,3,9,9,11,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2
top,ID15226,C8,B2,M1,Petrol,113Nm@4400rpm,88.50bhp@6000rpm,F8D Petrol Engine,No,Yes,No,Yes,No,Drum,Manual,Power,Yes,No,No,No,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes
freq,1,10973,14715,11914,16489,14281,14281,11914,32165,28481,35686,44958,28540,35686,30541,26703,27178,33326,33326,30456,25761,33998,33998,45912,27470,28968,33998,46582


* policy_id 去除
* onehot encoding: segment, fuel_type, steering_type, 
* target encoding：area_cluster, model, engine_type 
* max_torque 取出 Nm 數值
* max_power 取出 hp 數值
* rear_brakes_type 轉成 is_disc_brake 0, 1
* transmission_type 轉成 auto_transmission 0, 1
* 轉成 0, 1： is_esc, is_adjustable_steering, is_tpms, is_parking_sensors, is_parking_camera, is_front_fog_lights, is_rear_window_wiper, is_rear_window_washer, is_rear_window_defogger, is_brake_assist, is_power_door_locks, is_central_locking, is_power_steering, is_driver_seat_height_adjustable, is_day_night_rear_view_mirror, is_ecw, is_speed_alert

In [11]:
train_X['max_torque'] = train_X['max_torque'].str.split('Nm').str[0].astype(float)
train_X['max_power'] = train_X['max_power'].str.split('bhp').str[0].astype(float)

mask = (train_X['rear_brakes_type'] == "Disc")
train_X['is_disc_brake'] = 0 
train_X.loc[mask, 'is_disc_brake'] = 1

mask = (train_X['transmission_type'] == "Automatic")
train_X['auto_transmission'] = 0 
train_X.loc[mask, 'auto_transmission'] = 1
train_X = yes_no_01(train_X, 
    ['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
    'is_parking_camera', 'is_front_fog_lights', 'is_rear_window_wiper',
    'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist',
    'is_power_door_locks', 'is_central_locking', 'is_power_steering',
    'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert'])

# one-hot encode
train_X = one_hot_encode(train_X, ['make', 'segment', 'fuel_type', 'steering_type'])
# target encode
train_X, dict_t_area = target_encode(train_X, train_y, 'area_cluster')
train_X, dict_t_model = target_encode(train_X, train_y, 'model')
train_X, dict_t_engine = target_encode(train_X, train_y, 'engine_type')

train_X = train_X.drop(['policy_id', 'rear_brakes_type', 'transmission_type'], axis=1)

# ---- test ----
test_X['max_torque'] = test_X['max_torque'].str.split('Nm').str[0].astype(float)
test_X['max_power'] = test_X['max_power'].str.split('bhp').str[0].astype(float)

mask = (test_X['rear_brakes_type'] == "Disc")
test_X['is_disc_brake'] = 0 
test_X.loc[mask, 'is_disc_brake'] = 1

mask = (test_X['transmission_type'] == "Automatic")
test_X['auto_transmission'] = 0 
test_X.loc[mask, 'auto_transmission'] = 1
test_X = yes_no_01(test_X, 
    ['is_esc', 'is_adjustable_steering', 'is_tpms', 'is_parking_sensors',
    'is_parking_camera', 'is_front_fog_lights', 'is_rear_window_wiper',
    'is_rear_window_washer', 'is_rear_window_defogger', 'is_brake_assist',
    'is_power_door_locks', 'is_central_locking', 'is_power_steering',
    'is_driver_seat_height_adjustable', 'is_day_night_rear_view_mirror', 'is_ecw', 'is_speed_alert'])

# one-hot encode
test_X = one_hot_encode(test_X, ['make', 'segment', 'fuel_type', 'steering_type'])
# target encode
test_X['area_cluster'] = test_X['area_cluster'].replace(dict_t_area)
test_X['model'] = test_X['model'].replace(dict_t_model)
test_X['engine_type'] = test_X['engine_type'].replace(dict_t_engine)

test_X = test_X.drop(['policy_id', 'rear_brakes_type', 'transmission_type'], axis=1)

In [12]:
print(train_X.shape)
train_X.head(5)

(46874, 55)


,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,model,max_torque,max_power,engine_type,airbags,is_esc,is_adjustable_steering,is_tpms,is_parking_sensors,is_parking_camera,displacement,turning_radius,length,width,height,gross_weight,is_front_fog_lights,is_rear_window_wiper,is_rear_window_washer,is_rear_window_defogger,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,cylinder_4,gear_box_6,is_disc_brake,auto_transmission,make_3,make_1,make_2,make_5,make_4,segment_C2,segment_B1,segment_B2,segment_A,segment_C1,segment_Utility,fuel_type_Diesel,fuel_type_CNG,fuel_type_Petrol,steering_type_Power,steering_type_Electric,steering_type_Manual
0,0.696032,0.14,0.336538,0.052154,17804,0.064533,250.0,113.45,0.064533,6,1,1,1,1,1,1493,5.2,4300,1790,1635,1720,1,1,1,1,1,1,1,1,1,0,1,1,3,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1,1.068681,0.01,0.538462,0.069177,27003,0.060743,82.1,55.92,0.060743,2,0,0,0,1,0,998,4.7,3655,1620,1675,1340,0,0,0,0,0,1,1,1,0,0,1,1,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0
2,1.075288,0.05,0.346154,0.055200,6112,0.066142,113.0,88.50,0.066142,2,0,1,0,1,0,1197,4.8,3845,1735,1530,1335,1,0,0,0,1,1,1,1,1,1,1,1,2,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
3,1.208379,0.17,0.423077,0.074087,7788,0.064533,250.0,113.45,0.064533,6,1,1,1,1,1,1493,5.2,4300,1790,1635,1720,1,1,1,1,1,1,1,1,1,0,1,1,3,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
4,1.209388,0.02,0.317308,0.074087,7788,0.064533,250.0,113.45,0.064533,6,1,1,1,1,1,1493,5.2,4300,1790,1635,1720,1,1,1,1,1,1,1,1,1,0,1,1,3,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0


In [13]:
train_X.to_csv('../output/train_X.csv', index=False)
train_y.to_csv('../output/train_y.csv', index=False)

test_X.to_csv('../output/test_X.csv', index=False)
test_y.to_csv('../output/test_y.csv', index=False)